# 6-7. 프로젝트: 멋진 작사가 만들기


## 데이터 다운로드 

Step 1. 데이터 다운로드
먼저 아래 링크에서 Song Lyrics 데이터를 다운로드한다.  저장된 파일을 압축 해제한 후, 모든 txt 파일을 lyrics 폴더를 만들어 그 속에 저장해주세요!

[Song Lyrics](https://www.kaggle.com/paultimothymooney/poetry/data)

``` terminal
$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics       
#lyrics 폴더에 압축풀기
```


## 데이터 읽어 오기 

glob모듈을 사용해서 파일을 편리하게 읽어 올 수 있다.

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Aw yeah, yeah', "I mean there's a lot of women out here that's just, you know", 'They just defensive in so many things, man']


## 데이터 전 처리 

텐서 플로우에서 보통 자연어 데이터 전처리는 아래의 행동들을 지칭한다.

* 정규표현식을 이용한 corpus 생성
* tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
* tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

을 차례대로 진행하도록한다.


In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 되는것을 통해 확인

<start> this is sample sentence . <end>


preprocess_sentencee 함수를 통해 특수문자 등을 제외하고, 소문자 등만을 사용하게 하는등 정규표현식을 통해 깍아낸다.  

In [32]:
corpus = []
new_corpus = []
bad_corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]




['<start> aw yeah , yeah <end>',
 '<start> i mean there s a lot of women out here that s just , you know <end>',
 '<start> they just defensive in so many things , man <end>',
 '<start> you missing so many key nutrients in your life <end>',
 '<start> i , i just want you to stay healthy in these streets <end>',
 '<start> oh , no no <end>',
 '<start> somebody gotta give you some vitamin d , you know what i mean ? <end>',
 '<start> yeah , luda she got them handlebars <end>',
 '<start> gotta keep a tight grip on that donkey <end>',
 '<start> she be givin out battle scars <end>']

raw_corpus의 문장들중 공백인 문장은 제외하고  말하는 배역들이 나타나는 :로 끝나는 문장도 제외한다.                
그후 나머지 문장들에게 아까 생성한 preprocess_sentence를 적용한후 빈 list인 corpus에 넣는다.

In [35]:
for i in range((len(corpus))-1):
    if len(corpus[i].split(' ')) <= 15:
        new_corpus.append(corpus[i])
    else :
        bad_corpus.append(corpus[i])
        
print(bad_corpus[0])

<start> i mean there s a lot of women out here that s just , you know <end>


그후 15개 이하의 문장들을 걸러내기 위해서 for 문을 반복했다. 

In [36]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words= 12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen= 15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(new_corpus)

[[   2 1674   49 ...    0    0    0]
 [   2   38   32 ...    0    0    0]
 [   2    7 1278 ...    0    0    0]
 ...
 [   2   41    6 ...    0    0    0]
 [   2   31    7 ...    0    0    0]
 [   2  302    1 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f25f44c4cd0>


토큰화와 단어 사전을 만들어낸다.

In [37]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 1674   49    5   49    3    0    0    0    0    0    0    0    0]
[1674   49    5   49    3    0    0    0    0    0    0    0    0    0]


In [ ]:
dataset을 얻음으로써 데이터 다듬기 과정은 끝났습니다. tf.data.Dataset에서 제공하는 shuffle(), batch() 등 다양한 데이터셋 관련 기능을 손쉽게 이용할 수 있게 되었군요.

## 평가 데이터셋 분리


In [38]:
src_input.shape
tgt_input.shape

(156013, 14)

In [39]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,train_size = 0.8)

In [40]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


Source Train: (124810, 14)
Target Train: (124810, 14)


In [50]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)

        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

모델의 엠베딩 사이즈와 히든 사이즈를 상당히 늘려주었다. 

In [51]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [52]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.08942758e-04,  1.79483788e-04,  2.53361086e-05, ...,
          1.00675745e-04, -1.95242192e-05, -7.89281767e-05],
        [ 3.04634828e-04,  4.35290538e-04,  2.25911990e-05, ...,
          8.33552476e-05, -2.78933963e-04,  2.62175818e-05],
        [ 6.20262057e-04,  5.75228652e-04,  4.62264929e-04, ...,
          7.91750499e-05, -4.25172271e-04,  4.39110445e-04],
        ...,
        [ 1.02350104e-03,  1.38910161e-03, -1.75642912e-04, ...,
          2.44971598e-05, -1.51662724e-04, -1.89736520e-03],
        [ 1.03410054e-03,  1.57064700e-03, -1.94065389e-04, ...,
          2.90894328e-04, -5.33417158e-04, -2.29326705e-03],
        [ 1.02952821e-03,  1.70423870e-03, -1.74589804e-04, ...,
          5.51441452e-04, -9.49309615e-04, -2.62684398e-03]],

       [[ 2.08942758e-04,  1.79483788e-04,  2.53361086e-05, ...,
          1.00675745e-04, -1.95242192e-05, -7.89281767e-05],
        [ 1.17417358e-05,  2.01306379e-04, -9

## 모델 적합

In [53]:
model.summary()


Model: "text_generator_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_24 (LSTM)               multiple                  20979712  
_________________________________________________________________
lstm_25 (LSTM)               multiple                  33562624  
_________________________________________________________________
dense_9 (Dense)              multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 모델을 저장하려했으나 저장시간이 길어져서 제외함.
# # the save point
# checkpoint_dir = os.getenv('HOME')+'/ssac4/aiffel/lyricist/models/lyric'

# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
#                                                  save_weights_only=True,
#                                                  monitor='val_loss',
#                                                  mode='auto',
#                                                  save_best_only=True,
#                                                  verbose=1)
# print("✅")

✅


## 학습

In [54]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history  = model.fit(dataset,validation_data = (enc_val,dec_val), epochs=10)

Epoch 1/10
487/487 [==============================] - 218s 448ms/step - loss: 3.3159 - val_loss: 2.9447
Epoch 2/10
487/487 [==============================] - 200s 410ms/step - loss: 2.7781 - val_loss: 2.7040
Epoch 3/10
487/487 [==============================] - 200s 411ms/step - loss: 2.4930 - val_loss: 2.5294
Epoch 4/10
487/487 [==============================] - 200s 412ms/step - loss: 2.2132 - val_loss: 2.3953
Epoch 5/10
487/487 [==============================] - 201s 412ms/step - loss: 1.9435 - val_loss: 2.2924
Epoch 6/10
487/487 [==============================] - 201s 413ms/step - loss: 1.6921 - val_loss: 2.2194
Epoch 7/10
487/487 [==============================] - 210s 432ms/step - loss: 1.4716 - val_loss: 2.1713
Epoch 8/10
487/487 [==============================] - 214s 439ms/step - loss: 1.2909 - val_loss: 2.1499
Epoch 9/10
487/487 [==============================] - 205s 421ms/step - loss: 1.1536 - val_loss: 2.1531
Epoch 10/10
487/487 [==============================] - 204s 420m

## 가사 작성

실제 가사를 작성해보도록하였다, 

In [55]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [57]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)


'<start> i love you , i love you , i love you <end> '

'<start> i love you , i love you , i love you <end> '의 가사를 얻을 수 있었다.